In [1]:
# Construction of dataset

import os, itertools, time, pickle, sys, glob, requests
import subprocess
from xml.dom import minidom
from collections import Counter, OrderedDict
from operator import itemgetter
from nltk.corpus import wordnet
import tensorflow as tf
import tensorflow_hub as hub
from scipy import spatial
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np
import scipy.sparse as sp
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from math import ceil, exp
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import networkx as nx
import matplotlib.pyplot as plt
from sentence_transformers import models, SentenceTransformer
import sent2vec
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

%matplotlib inline  

In [2]:
flatten = lambda l: [item for sublist in l for item in sublist]

class Ontology():
    def __init__(self, ontology):
        self.ontology = ontology
        self.ontology_obj = minidom.parse(ontology)
        self.root = self.ontology_obj.documentElement
        self.construct_mapping_dict()
        
        self.parents_dict = {}
        self.subclasses = self.parse_subclasses()
        self.object_properties = self.parse_object_properties()
        self.data_properties = self.parse_data_properties()
        self.triples = self.parse_triples()
        self.classes = self.parse_classes()        
    
    def construct_mapping_dict(self):
        self.mapping_dict = {self.extract_ID(el, False): self.get_child_node(el, "rdfs:label")[0].firstChild.nodeValue for el in self.root.getElementsByTagName("owl:Class") if self.get_child_node(el, "rdfs:label")}
        self.mapping_dict_inv = {self.mapping_dict[key]: key for key in self.mapping_dict}
        return
        
    def get_child_node(self, element, tag):
        return [e for e in element._get_childNodes() if type(e)==minidom.Element and e._get_tagName() == tag]
        
    def has_attribute_value(self, element, attribute, value):
        return True if element.getAttribute(attribute).split("#")[-1] == value else False
    
    def get_subclass_triples(self):
        subclasses = self.get_subclasses()
        for (a,b,c) in subclasses:
            if c == "subclass_of" and a!="Thing" and b!="Thing":
                if b not in self.parents_dict:
                    self.parents_dict[b] = [a]
                else:
                    self.parents_dict[b].append(a)
        return [(b,a,c) for (a,b,c) in subclasses]
    
    def parse_triples(self, union_flag=0, subclass_of=True, data_prop=True):
        obj_props = self.object_properties
        if data_prop:
            data_props = self.data_properties
            props = obj_props + data_props
        else:
            props = obj_props
        all_triples = []
        for prop in props:
            domain_children = self.get_child_node(prop, "rdfs:domain")
            range_children = self.get_child_node(prop, "rdfs:range")
            domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children])
            range_prop = self.filter_null([self.extract_ID(el) for el in range_children])
            if not domain_children or not range_children:
                continue
            if not domain_prop:
                domain_prop = self.filter_null([self.extract_ID(el) for el in domain_children[0].getElementsByTagName("owl:Class")])
            if not range_prop:
                range_prop = self.filter_null([self.extract_ID(el) for el in range_children[0].getElementsByTagName("owl:Class")])
            if domain_prop and range_prop:
                if union_flag == 0:
                    all_triples.extend([(el[0], el[1], self.extract_ID(prop)) for el in list(itertools.product(domain_prop, range_prop))])
                else:
                    all_triples.append(("###".join(domain_prop), "###".join(range_prop), self.extract_ID(prop)))
        if subclass_of:
            all_triples.extend(self.get_subclass_triples())
        return list(set(all_triples))
    
    def get_triples(self, union_flag=0, subclass_of=True):
        return self.parse_triples(union_flag, subclass_of)

    def parse_subclasses(self, union_flag=0):
        subclasses = self.root.getElementsByTagName("rdfs:subClassOf")
        subclass_pairs = []
        for el in subclasses:
            inline_subclasses = self.extract_ID(el)
            if inline_subclasses:
                subclass_pairs.append((el, el.parentNode, "subclass_of"))
            else:
                level1_class = self.get_child_node(el, "owl:Class")
                if not level1_class:
                    restriction = el.getElementsByTagName("owl:Restriction")
                    if not restriction:
                        continue
                    prop = self.get_child_node(restriction[0], "owl:onProperty")
                    some_vals = self.get_child_node(restriction[0], "owl:someValuesFrom")
                    
                    if not prop or not some_vals:
                        continue
#                     print(self.extract_ID(el), "**", self.extract_ID(some_vals[0]), "**", self.extract_ID(prop[0]))
                    try:
                        if self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                            subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop[0])))
                        elif self.extract_ID(prop[0]) and not self.extract_ID(some_vals[0]):
                            class_vals = self.get_child_node(some_vals[0], "owl:Class")
                            subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop[0])))
                        elif not self.extract_ID(prop[0]) and self.extract_ID(some_vals[0]):
                            prop_vals = self.get_child_node(prop[0], "owl:ObjectProperty")
                            subclass_pairs.append((el.parentNode, some_vals[0], self.extract_ID(prop_vals[0])))
                        else:
                            prop_vals = self.get_child_node(prop[0], "owl:ObjectProperty")
                            class_vals = self.get_child_node(some_vals[0], "owl:Class")
                            subclass_pairs.append((el.parentNode, class_vals[0], self.extract_ID(prop_vals[0])))
                    except Exception as e:
                        print ("error", e)
                        continue
                else:
                    if self.extract_ID(level1_class[0]):
                        subclass_pairs.append((level1_class[0], el.parentNode, "subclass_of"))
                    else:
#                         level2classes = level1_class[0].getElementsByTagName("owl:Class")
#                         subclass_pairs.extend([(elem, el.parentNode, "subclass_of") for elem in level2classes if self.extract_ID(elem)])
                        continue
        return subclass_pairs
        
    def get_subclasses(self):
        subclasses = [(self.extract_ID(a), self.extract_ID(b), c) for (a,b,c) in self.subclasses]
        return [el for el in subclasses if el[0] and el[1] and el[2] and el[0]!="Thing" and el[1]!="Thing"]
    
    def filter_null(self, data):
        return [el for el in data if el]
    
    def extract_ID(self, element, check_coded = True):
        element_id = element.getAttribute("rdf:ID") or element.getAttribute("rdf:resource") or element.getAttribute("rdf:about")
        element_id = element_id.split("#")[-1]
        if len(list(filter(str.isdigit, element_id))) >= 3 and "_" in element_id and check_coded:
            return self.mapping_dict[element_id]
        return element_id.replace("UNDEFINED_", "").replace("DO_", "")
    
    def parse_classes(self):
        class_elems = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        subclass_classes = list(set(flatten([el[:-1] for el in self.triples])))
        return list(set(self.filter_null(class_elems + subclass_classes)))
    
    def get_classes(self):
        return self.classes
    
    def get_entities(self):
        entities = [self.extract_ID(el) for el in self.root.getElementsByTagName("owl:Class")]
        return list(set(self.filter_null(entities)))

    def parse_data_properties(self):
        data_properties = [el for el in self.get_child_node(self.root, 'owl:DatatypeProperty')]
        fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_data_properties = [el for el in fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        inv_fn_data_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_data_properties = [el for el in inv_fn_data_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "DatatypeProperty")]]
        return data_properties + fn_data_properties + inv_fn_data_properties
        
    def parse_object_properties(self):
        obj_properties = [el for el in self.get_child_node(self.root, 'owl:ObjectProperty')]
        fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:FunctionalProperty') if el]
        fn_obj_properties = [el for el in fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        inv_fn_obj_properties = [el for el in self.get_child_node(self.root, 'owl:InverseFunctionalProperty') if el]
        inv_fn_obj_properties = [el for el in inv_fn_obj_properties if type(el)==minidom.Element and 
            [el for el in self.get_child_node(el, "rdf:type") if 
             self.has_attribute_value(el, "rdf:resource", "ObjectProperty")]]
        return obj_properties + fn_obj_properties + inv_fn_obj_properties
    
    def get_object_properties(self):
        obj_props = [self.extract_ID(el) for el in self.object_properties]
        return list(set(self.filter_null(obj_props)))
    
    def get_data_properties(self):
        data_props = [self.extract_ID(el) for el in self.data_properties]
        return list(set(self.filter_null(data_props)))


In [20]:
USE_folder = "/home/vlead/USE"
alignment_folder = "../Anatomy/Alignments/"

# Load reference alignments 
def load_alignments(folder):
    alignments = []
    for f in os.listdir(folder):
        doc = minidom.parse(folder + f)
        ls = list(zip(doc.getElementsByTagName('entity1'), doc.getElementsByTagName('entity2')))
        alignments.extend([(a.getAttribute('rdf:resource'), b.getAttribute('rdf:resource')) for (a,b) in ls])
    return alignments

# model = SentenceTransformer('bert-large-nli-mean-tokens')

# def extract_huggingface_embeddings(words):
#     return model.encode(words)

def extract_huggingface_embeddings(words):
    try:
        embed = hub.KerasLayer(USE_folder)
    except Exception as e:
        !mkdir $USE_folder
        !curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed" | tar -zxvC $USE_folder
        embed = hub.KerasLayer(USE_folder)
        pass
    word_embeddings = embed(words)
    return word_embeddings.numpy()


# model_path = "/home/vlead/BioSentVec_PubMed_MIMICIII-bigram_d700.bin"
# model = sent2vec.Sent2vecModel()
# try:
#     model.load_model(model_path)
# except Exception as e:
#     print(e)
# print('model successfully loaded')

# stop_words = set(stopwords.words('english'))
# def preprocess_sentence(text):
#     text = text.replace('/', ' / ')
#     text = text.replace('.-', ' .- ')
#     text = text.replace('.', ' . ')
#     text = text.replace('\'', ' \' ')
#     text = text.lower()

#     tokens = [token for token in word_tokenize(text) if token not in punctuation and token not in stop_words]

#     return ' '.join(tokens)

# def extract_huggingface_embeddings(words):
#     return model.embed_sentences([preprocess_sentence(elem) for elem in words])

def cos_sim(a,b):
    try:
        sim = 1 - spatial.distance.cosine(a, b)
    except:
        sim = 0
    return sim


# reference_alignments = load_alignments(alignment_folder)

ra_anatomy_coded = load_alignments(alignment_folder)
ontologies_in_alignment = [["../Anatomy/Ontologies/mouse.owl", "../Anatomy/Ontologies/human.owl"]]

reference_alignments = []

for (ont1_name, ont2_name) in ontologies_in_alignment:
    ont1 = Ontology(ont1_name)
    ont2 = Ontology(ont2_name)
    for elem in ra_anatomy_coded:
        pre1, pre2 = elem[0].split("#")[0].split(".")[0].split("/")[-1], elem[1].split("#")[0].split(".")[0].split("/")[-1]
        elem1, elem2 = elem[0].split("#")[-1], elem[1].split("#")[-1]
        reference_alignments.append(( pre1 + "#" + ont1.mapping_dict[elem1], pre2 + "#" + ont2.mapping_dict[elem2]))

gt_mappings = reference_alignments
# gt_mappings.extend(ra_anatomy)


# ontologies_in_alignment = pickle.load(open("../data_generic.pkl", "rb"))[-1][:-1]


In [4]:
# Combinatorial mapping generation
all_mappings = []
for l in ontologies_in_alignment:
    ont1 = Ontology(l[0])
    ont2 = Ontology(l[1])
    
    ent1 = ont1.get_entities()
    ent2 = ont2.get_entities()
    
    obj1 = ont1.get_object_properties()
    obj2 = ont2.get_object_properties()
    
    data1 = ont1.get_data_properties()
    data2 = ont2.get_data_properties()

    mappings = list(itertools.product(ent1, ent2)) + list(itertools.product(obj1, obj2)) + list(itertools.product(data1, data2))

    ont_prefix1 = l[0].split("/")[-1].split(".")[0]
    ont_prefix2 = l[1].split("/")[-1].split(".")[0]
    
    all_mappings.extend([(ont_prefix1 + "#" + el[0], ont_prefix2 + "#" + el[1]) for el in mappings])
    

data = {mapping: False for mapping in all_mappings}
for mapping in set(gt_mappings):
    data[mapping] = True

In [5]:
# Abbrevation resolution preprocessing

abbreviations_dict = {}
final_dict = {}

for mapping in all_mappings:
    mapping = tuple([el.split("#")[1] for el in mapping])
    is_abb = re.search("[A-Z][A-Z]+", mapping[0])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[1].split("_")])
        if is_abb.group() in abbreviation:
            
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[1].split("_")[start:end])
            print ("left", mapping, abbreviation, fullform)
            
            rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

    is_abb = re.search("[A-Z][A-Z]+", mapping[1])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[0].split("_")])
        
        if is_abb.group() in abbreviation:
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[0].split("_")[start:end])
            print ("right", mapping, abbreviation, fullform)

            rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extract_huggingface_embeddings(keys)))

scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}


left ('optic II nerve', 'Internal_Intercostal_Muscle') IIM Internal_Intercostal
left ('optic II nerve', 'Inferior_Internal_Articular_Artery') IIAA Inferior_Internal
left ('optic II nerve', 'Internal_Iliac_Vein') IIV Internal_Iliac
left ('optic II nerve', 'Internal_Iliac_Artery_Branch') IIAB Internal_Iliac
left ('optic II nerve', 'Internal_Iliac_Artery') IIA Internal_Iliac
left ('abducens VI nerve', 'Velum_Interpositum_Cistern') VIC Velum_Interpositum
left ('trochlear IV nerve', 'Iliolumbar_Vein') IV Iliolumbar_Vein
left ('trochlear IV nerve', 'Superficial_Intracranial_Venous_System') SIVS Intracranial_Venous
left ('trochlear IV nerve', 'Intracranial_Venous_System') IVS Intracranial_Venous
left ('trochlear IV nerve', 'External_Iliac_Vein') EIV Iliac_Vein
left ('trochlear IV nerve', 'Ileal_Vein') IV Ileal_Vein
left ('trochlear IV nerve', 'Ileocecal_Valve') IV Ileocecal_Valve
left ('trochlear IV nerve', 'Interosseous_Vein') IV Interosseous_Vein
left ('trochlear IV nerve', 'Inferior_Vermia

left ('hippocampus CA1', 'Posterior_Carpal_Artery') PCA Carpal_Artery
left ('hippocampus CA1', 'External_Carotid_Artery_Branch') ECAB Carotid_Artery
left ('hippocampus CA1', 'Thymic_Epithelial_Cell_Capable_of_Differentiating_Towards_Both_Cortical_and_Medullary_Type') TECCODTBCAMT Cortical_and
left ('hippocampus CA1', 'Spinal_Cord_Arachnoid_Membrane') SCAM Cord_Arachnoid
left ('hippocampus CA1', 'External_Carotid_Artery') ECA Carotid_Artery
left ('hippocampus CA1', 'Costophrenic_Angle') CA Costophrenic_Angle
left ('hippocampus CA1', 'Medial_Circumflex_Artery') MCA Circumflex_Artery
left ('hippocampus CA1', 'Connective_and_Soft_Tissue_Cell') CASTC Connective_and
left ('hippocampus CA1', 'Coeliac_Artery') CA Coeliac_Artery
left ('hippocampus CA1', 'CA2_Field_of_the_Cornu_Ammonis') CFOTCA Cornu_Ammonis
left ('hippocampus CA1', 'Cerebral_Aqueduct') CA Cerebral_Aqueduct
left ('hippocampus CA1', 'Choroidal_Artery') CA Choroidal_Artery
left ('hippocampus CA1', 'CA4_Field_of_the_Cornu_Ammonis')

left ('cerebellum lobule II', 'Internal_Intercostal_Muscle') IIM Internal_Intercostal
left ('cerebellum lobule II', 'Inferior_Internal_Articular_Artery') IIAA Inferior_Internal
left ('cerebellum lobule II', 'Internal_Iliac_Vein') IIV Internal_Iliac
left ('cerebellum lobule II', 'Internal_Iliac_Artery_Branch') IIAB Internal_Iliac
left ('cerebellum lobule II', 'Internal_Iliac_Artery') IIA Internal_Iliac
left ('cerebellum lobule IV', 'Iliolumbar_Vein') IV Iliolumbar_Vein
left ('cerebellum lobule IV', 'Superficial_Intracranial_Venous_System') SIVS Intracranial_Venous
left ('cerebellum lobule IV', 'Intracranial_Venous_System') IVS Intracranial_Venous
left ('cerebellum lobule IV', 'External_Iliac_Vein') EIV Iliac_Vein
left ('cerebellum lobule IV', 'Ileal_Vein') IV Ileal_Vein
left ('cerebellum lobule IV', 'Ileocecal_Valve') IV Ileocecal_Valve
left ('cerebellum lobule IV', 'Interosseous_Vein') IV Interosseous_Vein
left ('cerebellum lobule IV', 'Inferior_Vermian_Artery') IVA Inferior_Vermian
le

In [6]:

def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

def parse(word):
    return flatten([el.split("_") for el in camel_case_split(word)])
    

extracted_elems = []

for ont_name in list(set(flatten(ontologies_in_alignment))):
    ont = Ontology(ont_name)
    entities = ont.get_entities()
    props = ont.get_object_properties() + ont.get_data_properties()
    triples = list(set(flatten(ont.get_triples())))
    extracted_elems.extend([ont_name.split("/")[-1].split(".")[0] + "#" + elem for elem in entities + props + triples])

extracted_elems = list(set(extracted_elems))
inp_anatomy = [" ".join(parse(word.split("#")[1])) for word in extracted_elems]

roman_regex = "^M{0,3}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$"
inp_anatomy = [" ".join([word.replace("/", " or ") for word in elem.split() 
           if not re.search(roman_regex, word) and not re.search("\d", word)]) for elem in inp_anatomy]

inp_resolved = []
for concept in inp_anatomy:
    for key in filtered_dict:
        concept = concept.replace(key, filtered_dict[key])
    final_list = []
    # Lowering case except in abbreviations
    for word in concept.split(" "):
        if not re.search("[A-Z][A-Z]+", word):
            final_list.append(word.lower())
        else:
            final_list.append(word)
    concept = " ".join(final_list)
    inp_resolved.append(concept)


print ("Total number of extracted unique classes and properties from entire RA set: ", len(extracted_elems))

extracted_elems = ["<UNK>"] + extracted_elems

embeds = np.array(extract_huggingface_embeddings(inp_resolved))
embeds = np.array([np.zeros(embeds.shape[1],)] + list(embeds))
# embeds = np.array([np.zeros(512,)] + list(extractUSEEmbeddings(inp_spellchecked)))
embeddings = dict(zip(extracted_elems, embeds))

emb_vals = list(embeddings.values())
emb_indexer = {key: i for i, key in enumerate(list(embeddings.keys()))}
emb_indexer_inv = {i: key for i, key in enumerate(list(embeddings.keys()))}


Total number of extracted unique classes and properties from entire RA set:  6055


In [7]:
def path_to_root(elem, ont_mappings):
    if elem not in ont_mappings or not ont_mappings[elem]:
        return []
    output = flatten([[e] + path_to_root(e, ont_mappings) for e in ont_mappings[elem]])
    return output

def get_one_hop_neighbours(ont, K=1):
    ont_obj = Ontology(ont)
    triples = ont_obj.get_triples()
    entities = [(a,b) for (a,b,c) in triples]
    neighbours_dict = {elem: [elem] for elem in list(set(flatten(entities)))}
    for e1, e2 in entities:
        neighbours_dict[e1].append(e2)
        neighbours_dict[e2].append(e1)
#     print (neighbours_dict)
    rootpath_dict = ont_obj.parents_dict
    rootpath_dict = {elem: path_to_root(elem, rootpath_dict) for elem in rootpath_dict}
    ont = ont.split("/")[-1].split(".")[0]
    
    for entity in neighbours_dict:
        if entity in rootpath_dict and len(rootpath_dict[entity]) > 0:
            neighbours_dict[entity].extend(rootpath_dict[entity])
        else:
            continue
    
    
#     prop_triples = ont_obj.get_triples(subclass_of=False)
#     neighbours_dict_props = {c: [c] for a,b,c in prop_triples}
#     for e1, e2, p in prop_triples:
#         neighbours_dict_props[p].extend([e1, e2])

    #neighbours_dict = {**neighbours_dict, **neighbours_dict_props}
    
    # for elem in ont_obj.get_entities() + ont_obj.get_object_properties() + ont_obj.get_data_properties():
    #     if elem not in neighbours_dict:
    #         neighbours_dict[elem] = [elem]

    neighbours_dict = {el: neighbours_dict[el][:1] + sorted(list(set(neighbours_dict[el][1:])))
                       for el in neighbours_dict}
#     neighbours_dict = {el: neighbours_dict[el][:23] for el in neighbours_dict if len( neighbours_dict[el]) > 2}
#     ont = ont.split("/")[-1].split(".")[0]
    neighbours_dict = {ont + "#" + el: [ont + "#" + e for e in neighbours_dict[el]] for el in neighbours_dict}
    return neighbours_dict

neighbours_dicts = {ont.split("/")[-1].split(".")[0]: get_one_hop_neighbours(ont) for ont in list(set(flatten(ontologies_in_alignment)))}
max_neighbours = np.max(flatten([[len(el[e]) for e in el] for el in neighbours_dicts.values()]))
neighbours_lens = {ont: {key: len(neighbours_dicts[ont][key]) for key in neighbours_dicts[ont]}
                   for ont in neighbours_dicts}
neighbours_dicts = {ont: {key: neighbours_dicts[ont][key] + ["<UNK>" for i in range(max_neighbours -len(neighbours_dicts[ont][key]))]
              for key in neighbours_dicts[ont]} for ont in neighbours_dicts}


data_items = data.items()
data_shuffled_t = [elem for elem in data_items if elem[1]]
data_shuffled_f = [elem for elem in data_items if not elem[1]][:400*len(data_shuffled_t)]

data_shuffled_train = data_shuffled_t[:int(0.9*len(data_shuffled_t))] + data_shuffled_f[:int(0.9*len(data_shuffled_f))]
data_shuffled_test = data_shuffled_t[int(0.9*len(data_shuffled_t)):] + data_shuffled_f[int(0.9*len(data_shuffled_f)):]

np.random.shuffle(data_shuffled_train)
np.random.shuffle(data_shuffled_test)

data_train = OrderedDict(data_shuffled_train)
data_test = OrderedDict(data_shuffled_test)

ontologies_in_alignment = [[el.split("/")[-1].split(".")[0] for el in ont] for ont in ontologies_in_alignment]
f = open("data_anatomy_use.pkl", "wb")
pickle.dump([data_train, data_test, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts, ontologies_in_alignment], f)

In [1]:
ontologies_in_alignment

NameError: name 'ontologies_in_alignment' is not defined

In [ ]:
model_path = "/home/vlead/BioSentVec_PubMed_MIMICIII-bigram_d700.bin"
model = sent2vec.Sent2vecModel()
try:
    model.load_model(model_path)
except Exception as e:
    print(e)
print('model successfully loaded')

In [ ]:

def cos_sim(a,b):
    return 1 - spatial.distance.cosine(a, b)

stop_words = set(stopwords.words('english'))
def preprocess_sentence(text):
    text = text.replace('/', ' / ')
    text = text.replace('.-', ' .- ')
    text = text.replace('.', ' . ')
    text = text.replace('\'', ' \' ')
    text = text.lower()

    tokens = [token for token in word_tokenize(text) if token not in punctuation and token not in stop_words]

    return ' '.join(tokens)

cos_sim(*model.embed_sentences([preprocess_sentence(elem) for elem in ["fastigial nucleus", "heart"]]))

In [10]:
# AML test
train_data, test_data = pickle.load(open("data_anatomy_bert.pkl", "rb"))[:2]
ontologies_in_alignment = [["../Anatomy/Ontologies/mouse.owl", "../Anatomy/Ontologies/human.owl"]]
test_data_t = [key for key in test_data if test_data[key]]
results = []
# all_ont_pairs = list(set([tuple([el.split("#")[0] for el in l]) for l in data.keys()]))
# for i in list(range(0, len(all_ont_pairs), 3)):
#     test_onto = all_ont_pairs[i:i+3]
for ont_pair in ontologies_in_alignment:
    a, b, c = ont_pair[0], ont_pair[1], ont_pair[0].split("/")[-1].split(".")[0] + "-" + ont_pair[1].split("/")[-1].split(".")[0]
    java_command = "java -jar AML_v3.1/AgreementMakerLight.jar -s " + a + " -t " + b + " -o AML-test-results/" + c + ".rdf -a"
    process = subprocess.Popen(java_command.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
print (os.listdir("AML-test-results/"))
pred_aml = load_alignments("AML-test-results/")

pred_aml_filt = []

for (ont1_name, ont2_name) in ontologies_in_alignment:
    ont1 = Ontology(ont1_name)
    ont2 = Ontology(ont2_name)
    for elem in pred_aml:
        pre1, pre2 = elem[0].split("#")[0].split(".")[0].split("/")[-1], elem[1].split("#")[0].split(".")[0].split("/")[-1]
        elem1, elem2 = elem[0].split("#")[-1], elem[1].split("#")[-1]
        pred_aml_filt.append(( pre1 + "#" + ont1.mapping_dict[elem1], pre2 + "#" + ont2.mapping_dict[elem2]))



# pred_aml_filt = [tuple([el.split("/")[-1] for el in key]) for key in pred_aml_filt]
tp = [elem for elem in pred_aml_filt if elem in test_data and test_data[elem]]
fn = [key for key in test_data_t if key not in set(pred_aml_filt)]
fp = [elem for elem in pred_aml_filt if elem in test_data and not test_data[elem]]


tp_len, fn_len, fp_len = len(tp), len(fn), len(fp)
precision = tp_len/(tp_len+fp_len)
recall = tp_len/(tp_len+fn_len)
f1score = 2 * precision * recall / (precision + recall)
f2score = 5 * precision * recall / (4 * precision + recall)
f0_5score = 1.25 * precision * recall / (0.25 * precision + recall)
print (precision, recall, f1score, f2score, f0_5score)

metrics = [precision, recall, f1score, f2score, f0_5score]
results.append(metrics)

_ = [os.remove(f) for f in glob.glob('AML-test-results/*')]
    
print ("Final Results:", np.mean(results, axis=0))

['mouse-human.rdf']
1.0 0.9013157894736842 0.9480968858131489 0.9194630872483223 0.9785714285714288
Final Results: [1.         0.90131579 0.94809689 0.91946309 0.97857143]


In [12]:
fn

[('mouse#adrenal gland zona glomerulosa', 'human#Glomeruloza_Zone'),
 ('mouse#myelencephalon', 'human#Medulla_Oblongata'),
 ('mouse#osseus labyrinth vestibule', 'human#Vestibule'),
 ('mouse#mammary gland fluid/secretion', 'human#Breast_Fluid_or_Secretion'),
 ('mouse#corpora quadrigemina', 'human#Quadrigeminal_Body'),
 ('mouse#spleen periarteriolar lymphatic sheath',
  'human#Periarteriolar_Lymphoid_Sheath'),
 ('mouse#cerebellar layer', 'human#Cortical_Cell_Layer_of_the_Cerebellum'),
 ('mouse#eye sebaceous gland', 'human#Sebaceous_Gland_of_the_Eyelash'),
 ('mouse#oral epithelium', 'human#Oral_Cavity_Epithelium'),
 ('mouse#ovary follicle fluid', 'human#Ovarian_Follicle_Fluid'),
 ('mouse#telencephalon', 'human#Supratentorial_Brain'),
 ('mouse#medial cuneiform', 'human#Internal_Cuneiform_Bone_of_the_Foot'),
 ('mouse#coccygeal vertebra', 'human#Coccyx'),
 ('mouse#hippocampus CA1', 'human#CA1_Field_of_the_Cornu_Ammonis'),
 ('mouse#obturator externus', 'human#Obturator_Externus_Muscle')]

In [ ]:
[elem for elem in pred_aml_filt if "#" not in elem[0] or "#" not in elem[1]]

In [ ]:
f = open("data_unhas.pkl", "wb")
pickle.dump([data, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts, ontologies_in_alignment], f)


In [ ]:
def count_non_unk(elem):
    return len([l for l in elem if l!="<UNK>"])
neighbours_dicts = {ont: {el: neighbours_dicts[ont][el][:int(sys.argv[1])] for el in neighbours_dicts[ont]
       if count_non_unk(neighbours_dicts[ont][el]) > int(sys.argv[2])} for ont in neighbours_dicts}

In [8]:
len(data), len(all_mappings)

(9066182, 9066182)

In [46]:
train_data_t = [key for key in train_data if train_data[key]]
excluded_aml = [key for key in test_data_t + train_data_t if key not in set(pred_aml_filt)]

In [ ]:
import requests

url = "https://montanaflynn-spellcheck.p.rapidapi.com/check/"

headers = {
    'x-rapidapi-host': "montanaflynn-spellcheck.p.rapidapi.com",
    'x-rapidapi-key': "9965b01207msh06291e57d6f2c55p1a6a16jsn0fb016da4a62"
    }

# inp_spellchecked = []
for concept in inp[731:]:
    querystring = {"text": concept}
    response = requests.request("GET", url, headers=headers, params=querystring).json()
    if response["suggestion"] != concept:
        resolved = str(concept)
        for word in response["corrections"]:
            if not re.search("[A-Z][A-Z]+", concept):
                resolved = resolved.replace(word, response["corrections"][word][0])
        
        inp_spellchecked.append(resolved)
        print (concept, resolved)
    else:
        inp_spellchecked.append(concept)




In [ ]:
querystring = {"text": "technically Organised By"}
response = requests.request("GET", url, headers=headers, params=querystring)
response.json()

In [ ]:
data_train, data_test, emb_indexer, emb_indexer_inv, emb_vals, gt_mappings, neighbours_dicts, ontologies_in_alignment = pickle.load(open("data_anatomy_sent2vec.pkl", "rb"))

In [ ]:
# fn_spellchecked, fp_spellchecked = [dict(el) for el in pickle.load(open("test_v2.pkl", "rb"))]
# fn_baseline, fp_baseline = [dict(el) for el in pickle.load(open("test_best.pkl", "rb"))]
# fn_unhas, fp_unhas = [dict(el) for el in pickle.load(open("test_unhas.pkl", "rb"))]
# fn_resolved, fp_resolved = [dict(el) for el in pickle.load(open("test_resolved.pkl", "rb"))]

fn_dict, fp_dict = {}, {}
def create_comparison_file(file, idx):
    fn, fp = [dict(el) for el in pickle.load(open(file, "rb"))]
    
    for key in fn:
        if key in fn_dict:
            fn_dict[key][idx] = fn[key]
        else:
            fn_dict[key] = ["N/A" for i in range(2)]
            fn_dict[key][idx] = fn[key]
    
    for key in fp:
        if key in fp_dict:
            fp_dict[key][idx] = fp[key]
        else:
            fp_dict[key] = ["N/A" for i in range(2)]
            fp_dict[key][idx] = fp[key]
    

create_comparison_file("test_best.pkl", 0)
create_comparison_file("test_unhas.pkl", 1)

open("fn - comparison.tsv", "w+").write("\n".join(["\t".join([str(el) for el in flatten(el)]) for el in fn_dict.items()]))
open("fp - comparison.tsv", "w+").write("\n".join(["\t".join([str(el) for el in flatten(el)]) for el in fp_dict.items()]))

In [43]:
fn_spellchecked, fp_spellchecked = [dict(el) for el in pickle.load(open("test_anatomy_use28_1.pkl", "rb"))]

In [44]:
fn_spellchecked

{('mouse#cerebellar layer',
  'human#Cortical_Cell_Layer_of_the_Cerebellum'): 0.8012549559910933,
 ('mouse#vertebra dorsal arch',
  'human#Arch_of_the_Vertebra'): 0.8190866110128275,
 ('mouse#iliopsoas', 'human#Iliopsoas_Muscle'): 0.7605249355878596,
 ('mouse#retina inner nuclear layer',
  'human#Inner_Nuclear_Layer'): 0.8656842872069336,
 ('mouse#stomach greater curvature',
  'human#Greater_Curvature'): 0.7226314051381755,
 ('mouse#corpora quadrigemina',
  'human#Quadrigeminal_Body'): 0.45310238770181666,
 ('mouse#palatine gland', 'human#Palatine_Salivary_Gland'): 0.8566996196072939,
 ('mouse#lunate', 'human#Lunate_Bone'): 0.7091920887814754,
 ('mouse#crypt of lieberkuhn',
  'human#Crypts_of_the_Lieberkuhn'): 0.7354369438482792,
 ('mouse#kidney interstitium',
  'human#Renal_Interstitial_Tissue'): 0.8072747064853303,
 ('mouse#arrector pili smooth muscle',
  'human#Erector_Muscle_of_the_Hair'): 0.7417348746968164,
 ('mouse#tooth substance', 'human#Tooth_Tissue'): 0.8003664647008165,
 ('

In [ ]:
abbreviations_dict = {}
final_dict = {}

for mapping in all_mappings:
    mapping = tuple([el.split("#")[1] for el in mapping])
    is_abb = re.search("[A-Z][A-Z]+", mapping[0])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[1].split("_")])
        if is_abb.group() in abbreviation:
            
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[1].split("_")[start:end])
            print ("left", mapping, abbreviation, fullform)
            
            rest_first = " ".join([el for el in mapping[0].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[1].split("_")[:start] + mapping[1].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

    is_abb = re.search("[A-Z][A-Z]+", mapping[1])
    if is_abb:
        abbreviation = "".join([el[0].upper() for el in mapping[0].split("_")])
        
        if is_abb.group() in abbreviation:
            start = abbreviation.find(is_abb.group())
            end = start + len(is_abb.group())
            fullform = "_".join(mapping[0].split("_")[start:end])
            print ("right", mapping, abbreviation, fullform)

            rest_first = " ".join([el for el in mapping[1].replace(is_abb.group(), "").split("_") if el]).lower()
            rest_second = " ".join(mapping[0].split("_")[:start] + mapping[0].split("_")[end:])
            if is_abb.group() not in final_dict:
                final_dict[is_abb.group()] = [(fullform, rest_first, rest_second)]
            else:
                final_dict[is_abb.group()].append((fullform, rest_first, rest_second))

keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))

scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)

resolved_dict = {key: scored_dict[key][0] for key in scored_dict}
filtered_dict = {key: " ".join(resolved_dict[key][0].split("_")) for key in resolved_dict if resolved_dict[key][-1] > 0.9}
inp_resolved = []
for concept in inp:
    for key in filtered_dict:
        concept = concept.replace(key, filtered_dict[key])
    inp_resolved.append(concept)
inp_resolved

In [ ]:
keys = [el for el in list(set(flatten([flatten([tup[1:] for tup in final_dict[key]]) for key in final_dict]))) if el]
abb_embeds = dict(zip(keys, extractUSEEmbeddings(keys)))


In [ ]:
reference_alignments

In [ ]:
len([key for key in data_train if data_train[key]]), len([key for key in data_test if data_test[key]])

In [ ]:
scored_dict = {}
for abbr in final_dict:
    sim_list = [(tup[0], tup[1], tup[2], cos_sim(abb_embeds[tup[1]], abb_embeds[tup[2]])) if tup[1] and tup[2]
                else (tup[0], tup[1], tup[2], 0) for tup in final_dict[abbr]]
    scored_dict[abbr] = sorted(list(set(sim_list)), key=lambda x:x[-1], reverse=True)


In [ ]:
inp_case_handled = []
for concept in inp:
    final_list = []
    for word in concept.split(" "):
        if not re.search("[A-Z][A-Z]+", concept):
            final_list.append(word.lower())
        else:
            final_list.append(word)
    case_resolved = " ".join(final_list)
    inp_case_handled.append(case_resolved)
    
inp_case_handled

In [ ]:
Ontology("conference_ontologies/conference.owl").triples

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [ ]:
# from transformers import XLNetTokenizer, XLNetModel
# import torch
# import scipy
# import torch.nn as nn
# import torch.nn.functional as F

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

input_ids = torch.tensor(tokenizer.encode("fastigial nucleus", add_special_tokens=True)).unsqueeze(0)
outputs = model(input_ids)
last_hidden_states = outputs[0].mean(1)

input_ids = torch.tensor(tokenizer.encode("femur", add_special_tokens=True)).unsqueeze(0) 

outputs1 = model(input_ids)
last_hidden_states1 = outputs1[0].mean(1)

cos = nn.CosineSimilarity(dim=1, eps=1e-6)
cos(last_hidden_states, last_hidden_states1)

In [28]:
# model_path = "/home/vlead/BioSentVec_PubMed_MIMICIII-bigram_d700.bin"
# model = sent2vec.Sent2vecModel()
# try:
#     model.load_model(model_path)
# except Exception as e:
#     print(e)
# print('model successfully loaded')

stop_words = set(stopwords.words('english'))
def preprocess_sentence(text):
    text = text.replace('/', ' / ')
    text = text.replace('.-', ' .- ')
    text = text.replace('.', ' . ')
    text = text.replace('\'', ' \' ')
    text = text.lower()

    tokens = [token for token in word_tokenize(text) if token not in punctuation and token not in stop_words]

    return ' '.join(tokens)

def extract_huggingface_embeddings(words):
    return model.embed_sentences([preprocess_sentence(elem) for elem in words])


cos_sim(*extractUSEEmbeddings(["adrenal gland zona reticularis", "Reticularis Zone"])), cos_sim(*extract_huggingface_embeddings(["adrenal gland zona reticularis", "Reticularis Zone"]))

0.35638532042503357

In [22]:
def extractUSEEmbeddings(words):
    try:
        embed = hub.KerasLayer(USE_folder)
    except Exception as e:
        !mkdir $USE_folder
        !curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/5?tf-hub-format=compressed" | tar -zxvC $USE_folder
        embed = hub.KerasLayer(USE_folder)
        pass
    word_embeddings = embed(words)
    return word_embeddings.numpy()

In [51]:
use, bio2vec = 0, 0
final_list = []
for pair in fn_spellchecked:
    pair_proc = [" ".join(parse(el.split("#")[1])).lower() for el in pair]
    use_score = cos_sim(embeddings[pair[0]], embeddings[pair[1]])
    bio2vec_score = cos_sim(*extract_huggingface_embeddings(list(pair_proc)))
    use += use_score
    bio2vec += bio2vec_score
    final_list.append([pair[0], pair[1], fn_spellchecked[pair], use_score, bio2vec_score, pair in excluded_aml, ""])
open("results_anatomy_use_fn.tsv", "w+").write("\n".join(["\t".join([str(col) for col in entry]) for entry in final_list]))
print ("Total", use, bio2vec)

Total 17.165284897635363 17.700582340359688


In [52]:
use, bio2vec = 0, 0
final_list = []
for pair in fp_spellchecked:
    pair_proc = [" ".join(parse(el.split("#")[1])).lower() for el in pair]
    use_score = cos_sim(embeddings[pair[0]], embeddings[pair[1]])
    bio2vec_score = cos_sim(*extract_huggingface_embeddings(list(pair_proc)))
    use += use_score
    bio2vec += bio2vec_score
    final_list.append([pair[0], pair[1], fp_spellchecked[pair], use_score, bio2vec_score, pair in excluded_aml, ""])
open("results_anatomy_use_fp.tsv", "w+").write("\n".join(["\t".join([str(col) for col in entry]) for entry in final_list]))
print ("Total", use, bio2vec)

Total 12.82707794533874 11.156346499919891
